In [ ]:
import os
import time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from os import listdir
from os.path import isfile, join

from lxml import etree



# <<<---configuration--->>>

copy model to 'current_model' folder

copy images to 'img_for_labeling' folder

# <<<---configuration--->>>

In [ ]:
model_path = os.path.join('Tensorflow', 'automated_labelling','current_model')
MODEL_NAME = [name for name in os.listdir(model_path) if os.path.isdir(model_path)]

CUSTOM_MODEL_NAME = MODEL_NAME[0] 
LABEL_MAP_NAME = 'label_map.pbtxt'

print('Model: ',CUSTOM_MODEL_NAME)

In [ ]:
paths = {
    'MODEL_PATH': os.path.join('Tensorflow', 'automated_labelling','current_model'),
    'IMG_BEFORE_LAB_PATH': os.path.join('Tensorflow', 'automated_labelling','img_for_labeling'),
    'IMG_AFTER_PATH': os.path.join('Tensorflow', 'automated_labelling','label_img'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'automated_labelling','current_model',CUSTOM_MODEL_NAME),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'automated_labelling','current_model',CUSTOM_MODEL_NAME,'dev','annotations'),
 }

creating folders from path:

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join(paths['MODEL_PATH'], CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

Getting latest checkpoint from models folder:

In [ ]:
file_names_in_model_dir = os.path.join(paths['MODEL_PATH'],CUSTOM_MODEL_NAME)
ckpt_files = []
all_files_from_model_dir = [f for f in listdir(file_names_in_model_dir) if isfile(join(file_names_in_model_dir, f))] #lista nazw plikow
for i in all_files_from_model_dir :
    if i[-1]=='x': (ckpt_files.append(i))
print(ckpt_files)
numbers = []
temp_num = ""
# extract number of ckpt files:
for one_name in ckpt_files:
    for c in one_name:
        if c.isdigit(): # finding numbers in string
            temp_num = temp_num + c
    numbers.append(int(temp_num))
    temp_num=""
    max_num = max(numbers)#largest number from list
latest_checkpoint_name = "ckpt-"+str(max_num)  #name of latest checkpoint
print(CUSTOM_MODEL_NAME,'---------->',latest_checkpoint_name)

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'],latest_checkpoint_name)).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
files = [f for f in listdir(paths['IMG_BEFORE_LAB_PATH']) if isfile(join(paths['IMG_BEFORE_LAB_PATH'], f))] #lista nazw plikow
files.sort()
jpg_images = [] #list for image names
for i in files:
    if i[-1]=='g':
        jpg_images.append(i)
# print(jpg_images)

for image_name in jpg_images:
    IMAGE_PATH = os.path.join(paths['IMG_BEFORE_LAB_PATH'], image_name)
    
    img = cv2.imread(IMAGE_PATH)
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.8,
                agnostic_mode=False)
    
    ymin = int(detections['detection_boxes'][0][0]*480)
    xmin = int(detections['detection_boxes'][0][1]*640)
    ymax = int(detections['detection_boxes'][0][2]*480)
    xmax = int(detections['detection_boxes'][0][3]*640)
    
    
    obj_with_name = category_index[detections['detection_classes'][0]+label_id_offset] #wydobycie danej sekcji z nazwa
    finded_object_name = obj_with_name['name']
    
    
    root = etree.Element("annotation")
    folder = etree.SubElement(root, "folder").text = 'unimportant'
    filename = etree.SubElement(root, "filename").text = image_name
    path = etree.SubElement(root, "path").text = 'unimportant'
    source = etree.SubElement(root, "source")
    source_database = etree.SubElement(source, "database").text = 'Unknown'
    size = etree.SubElement(root, "size")
    size_width = etree.SubElement(size, "width").text = '640'
    size_height = etree.SubElement(size, "height").text = '480'
    size_depth = etree.SubElement(size, "depth").text = '3'
    segmented = etree.SubElement(root, "segmented").text = '0'
    object_ = etree.SubElement(root, "object")
    object_name = etree.SubElement(object_, "name").text = finded_object_name
    object_pose = etree.SubElement(object_, "pose").text = 'Unspecified'
    object_truncated = etree.SubElement(object_, "truncated").text = '0'
    object_difficult = etree.SubElement(object_, "difficult").text = '0'
    object_bndbox = etree.SubElement(object_, "bndbox")
    object_bndbox_xmin = etree.SubElement(object_bndbox, "xmin").text = str(xmin)
    object_bndbox_ymin = etree.SubElement(object_bndbox, "ymin").text = str(ymin)
    object_bndbox_xmax = etree.SubElement(object_bndbox, "xmax").text = str(xmax)
    object_bndbox_ymax = etree.SubElement(object_bndbox, "ymax").text = str(ymax)
    
    tree_to_save = etree.ElementTree(root)
    XML_NAME_FILE = image_name.rstrip('jpg')+"xml"
    XML_PATH = os.path.join(paths['IMG_BEFORE_LAB_PATH'], XML_NAME_FILE)
    tree_to_save.write(XML_PATH, pretty_print=True)
    print(XML_NAME_FILE)
    
    
    
    
    coordinate = ['Ymin', 'Xmin', 'Ymax', 'Xmax'] 
    
    print(image_name,"|","class:",detections['detection_classes'][0],"| detection:",detections['detection_scores'][0],"|")
    print(detections['detection_boxes'][0])
    print(detections['detection_boxes'][0]*480)
    print(coordinate[0],int(detections['detection_boxes'][0][0]*480))
    print(coordinate[1],int(detections['detection_boxes'][0][1]*640))
    print(coordinate[2],int(detections['detection_boxes'][0][2]*480))
    print(coordinate[3],int(detections['detection_boxes'][0][3]*640))
    plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    plt.show()


    
